<a href="https://colab.research.google.com/github/Kazantsev-Alexey/ML_DS/blob/main/More_about_SQL_raw_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## План занятия

- Повторим как создавать БД при помощи SQLalchemy
- Разберем новые операторы в SQL
- Разберем подробнее ORM (объектно-реляционное отображение) в SQLalchemy

https://colab.research.google.com/drive/1gLP8_I9RdARVUoOwErCdyLv9V-xgqqoE?usp=sharing

(задание из презентации)

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
prices = np.random.permutation(np.arange(10))
prices

array([2, 5, 8, 1, 6, 7, 3, 4, 9, 0])

In [4]:
k = 3
for i in range(len(prices)):
    n_leq = 0
    for j in range(len(prices)):
        if prices[j] >= prices[i]:
            n_leq += 1
    if n_leq == k:
        print(prices[i])
        break

7


# SQLAlchemy. Recap

In [5]:
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

In [6]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)


In [7]:
Base = declarative_base()

class Person(Base):
    __tablename__= 'person'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)
    
class Address(Base):
    __tablename__ = 'address'

    id = Column(Integer, primary_key=True)
    country = Column(String(250))
    state = Column(String(250))
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250))
    person_id = Column(Integer, ForeignKey('person.id'))
    person = relationship(Person)

engine = create_engine('sqlite:///sqlalchemy_example.db')

connection = engine.connect()

Base.metadata.create_all(engine)

In [8]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [9]:
DBSession = sessionmaker(bind=engine)
session = DBSession()

new_person = Person(name='Nawin')
session.add(new_person)
session.commit()

new_address = Address(post_code='48103',country = 'USA', state = 'WA', street_name ='West Huron Street', street_number ='1', person=new_person)
session.add(new_address)
session.commit()

In [10]:
engine.table_names()

['address', 'person']

In [11]:
from sqlalchemy import select 

query = select([Person])

display_results(query)

,id,name
0,1,Nawin
1,2,Moshe
2,3,Lisa
3,4,Nika
4,5,Max
5,6,Dan
6,7,Alex
7,8,Nawin


In [12]:
query = select([Address])
display_results(query)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,Russia,Moscow,Pervomay Street,42,141700,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,USA,CA,Green Street,104,655689,4
4,5,USA,WA,West Huron Street,1,48103,8


Добавим несколько людей с разными адресами

In [13]:
from sqlalchemy import exists

def record_exists(session, name):
    return session.query(exists().where(Person.name == name)).scalar()

In [14]:
record_exists(session, 'Nawin')

True

In [15]:
record_exists(session, 'Lisa')

True

In [16]:
names = ['Moshe', 'Lisa', 'Nika']
adress_info = [('141700', 'Russia', 'Moscow', 'Pervomay Street', '42'), #Moshe adress
               ('141700', 'Russia', 'Moscow', 'Pervomay Street', '42'), #Lisa adress
               ('655689', 'USA', 'CA', 'Green Street', '104')] #Nika adress

In [17]:
for i, name, adress in zip(range(len(names)), names, adress_info):
    if not record_exists(session, name):
        new_person = Person(name=name)
        new_address = Address(country=adress[1],state=adress[2], street_number=adress[4], post_code=adress[0],street_name =adress[3], person=new_person)
        session.add(new_person)
        session.add(new_address)
    else:
        print(f"{name} Already exists, skipping...")

session.commit()

Moshe Already exists, skipping...
Lisa Already exists, skipping...
Nika Already exists, skipping...


Проверим теперь таблицу

In [18]:
query = select([Person])
display_results(query)

,id,name
0,1,Nawin
1,2,Moshe
2,3,Lisa
3,4,Nika
4,5,Max
5,6,Dan
6,7,Alex
7,8,Nawin


In [19]:
query = select([Address])
display_results(query)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,Russia,Moscow,Pervomay Street,42,141700,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,USA,CA,Green Street,104,655689,4
4,5,USA,WA,West Huron Street,1,48103,8


Другой способ добавления с использованием **add_all()**

In [20]:
session.add_all([Person(name='Max'),
                 Person(name='Dan'),
                 Person(name='Alex')])
session.commit()

In [21]:
query = select([Person.id, Person.name])
display_results(query)

,id,name
0,1,Nawin
1,2,Moshe
2,3,Lisa
3,4,Nika
4,5,Max
5,6,Dan
6,7,Alex
7,8,Nawin
8,9,Max
9,10,Dan


In [22]:
query = select([Address])
display_results(query)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,Russia,Moscow,Pervomay Street,42,141700,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,USA,CA,Green Street,104,655689,4
4,5,USA,WA,West Huron Street,1,48103,8


# Синтаксис 'чистых' SQL запросов. Повторение + новый материал

## Create

In [23]:
engine.table_names()

['address', 'person']

In [24]:
connection.execute('''
CREATE TABLE friends (
    id_ INT UNSIGNED AUTO_INCREMENT,
    friendName VARCHAR(50),
    phone VARCHAR(50),
    addressLine1 VARCHAR(50),
    addressLine2 VARCHAR(50),
    city VARCHAR(50),
    state VARCHAR(50),
    postalCode VARCHAR(50),
    country VARCHAR(50),
    PRIMARY KEY(id_)
);
''')

In [25]:
engine.table_names()

['address', 'friends', 'person']

In [26]:
result = connection.execute("select * from friends")
for row in result:
    print("Friend info", row)

In [27]:
sql = 'SELECT * from friends;'
pd.read_sql(sql, connection)

,id_,friendName,phone,addressLine1,addressLine2,city,state,postalCode,country


In [28]:
pd.read_sql('SELECT * from person;', connection)

,id,name,Rows_num
0,1,Nawin,7
1,2,Moshe,7
2,3,Lisa,7
3,4,Nika,7
4,5,Max,7
5,6,Dan,7
6,7,Alex,7
7,8,Nawin,7
8,9,Max,7
9,10,Dan,7


In [29]:
result = connection.execute(
'''SELECT
    country,
    state,
    post_code
FROM 
    address
WHERE 
    country = 'USA' AND 
    state = 'CA';
''')

for row in result:
    print("adress info", row)
pd.read_sql(result)

adress info ('USA', 'CA', '655689')


TypeError: ignored

## Insert into

In [30]:
connection.execute(
"""
INSERT INTO friends (
    id_,
    country,
    state,
    postalCode
)
VALUES (1, "UK", "London", 5678)
"""
)

In [ ]:
pd.read_sql('SELECT * from friends;', connection)

## Update

In [31]:
connection.execute(
'''
UPDATE friends 

SET
    friendName = "Mike"
WHERE 
    country = "UK"
''')

In [ ]:
pd.read_sql('SELECT * from friends;', connection)

## Insert into select

In [32]:
connection.execute(
'''
INSERT INTO friends (
    country,
    state,
    postalCode
)
SELECT
    country,
    state,
    post_code
FROM 
    address
WHERE 
    country = 'USA' AND 
    state = 'CA';
''')

In [33]:
pd.read_sql('SELECT * from friends;', connection)

,id_,friendName,phone,addressLine1,addressLine2,city,state,postalCode,country
0,1.0,Mike,None,None,None,None,London,5678,UK
1,NaN,None,None,None,None,None,CA,655689,USA


## Drop

In [34]:
connection.execute('DROP TABLE friends;')

In [35]:
engine.table_names()

['address', 'person']

## Create view

CREATE VIEW создаст представление по заданному вами условию

In [36]:
connection.execute('''
CREATE VIEW test_view AS
    SELECT 
        street_name, 
        street_number
    FROM
        address
    ORDER BY street_number;
''')

In [37]:
engine.table_names()

['address', 'person']

Но в pandas таблицы и представления разделяют одно пространство имен 

In [38]:
pd.read_sql('SELECT * from test_view;', connection)

,street_name,street_number
0,West Huron Street,1
1,West Huron Street,1
2,Green Street,104
3,Pervomay Street,42
4,Pervomay Street,42


In [39]:
connection.execute('''
drop view test_view;
''')

## Case

Выражение MySQL CASE - это структура потока управления, которая позволяет добавлять логику if-else к запросу. Вообще говоря, вы можете использовать выражение CASE в любом месте, которое допускает допустимое выражение, например, предложения SELECT, WHERE и ORDER BY.

In [40]:
pd.read_sql('SELECT * from address;', connection)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,Russia,Moscow,Pervomay Street,42,141700,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,USA,CA,Green Street,104,655689,4
4,5,USA,WA,West Huron Street,1,48103,8


In [42]:
pd.read_sql('''
SELECT 
    
    country `Страна`,
    CASE country
		WHEN "Russia" THEN 'Живет в России'
        WHEN "USA" THEN 'Живет в США'
        ELSE 'Живет в другой стране'
	end `Страна проживания`
FROM
    (SELECT 
		country
	FROM
		address
	INNER JOIN person
		ON  address.person_id = person.id) as cte
ORDER BY country;
''', connection)

,Страна,Страна проживания
0,Russia,Живет в России
1,Russia,Живет в России
2,USA,Живет в США
3,USA,Живет в США
4,USA,Живет в США


Вопрос: Почему в данном случае плохо использовать следующий синтаксис "JOIN ... USING (id)" ?

Как это исправить?

In [43]:
pd.read_sql('''
SELECT 
    name 'Имя',
    country 'Страна',
    CASE country
		WHEN "Russia" THEN 'Живет в России'
        WHEN "USA" THEN 'Живет в США'
        ELSE 'Живет в другой стране'
	end 'Страна проживания'
FROM
    (SELECT 
		country,
        name
	FROM
		address
	INNER JOIN person
		USING (id))
ORDER BY country;
''', connection)

,Имя,Страна,Страна проживания
0,Moshe,Russia,Живет в России
1,Lisa,Russia,Живет в России
2,Nawin,USA,Живет в США
3,Nika,USA,Живет в США
4,Max,USA,Живет в США


In [ ]:
# TODO

ЗАДАНИЕ

Добавить еще один столбец - Имя человека

In [ ]:
#TODO

In [ ]:
# TODO

ДОП ЗАДАНИЕ**

Если человек живет в Москве, то вывести его имя, улицу и почтовый код

In [44]:
pd.read_sql('''SELECT name  'Имя',
    post_code 'почтовый код',
    street_name 'улица' FROM(
SELECT name, post_code, street_name, state
    FROM address
	INNER JOIN person
		ON  address.person_id = person.id
    WHERE state = 'Moscow') ''', connection)

,Имя,почтовый код,улица
0,Moshe,141700,Pervomay Street
1,Lisa,141700,Pervomay Street


In [45]:
rnum=pd.read_sql('''select count(name) as name from person''', connection)
row_num=rnum.iloc[0]['name']
connection.execute(f'''alter table person add column Rows_num default ({row_num})''')
# при попытке создать для дефолт значение из подзапроса селект возвращается ошибка
#о том что значение не является константным
#поэтому пришлось вставлять данные через переменную 

OperationalError: ignored

In [46]:
pd.read_sql(''' select * from person ''', connection)

,id,name,Rows_num
0,1,Nawin,7
1,2,Moshe,7
2,3,Lisa,7
3,4,Nika,7
4,5,Max,7
5,6,Dan,7
6,7,Alex,7
7,8,Nawin,7
8,9,Max,7
9,10,Dan,7


In [ ]:
pd.read_sql('''select* from person''',connection)

# Работа с датами

## Date

MySQL DATE - один из пяти временных типов данных, используемых для управления значениями даты. MySQL использует формат гггг-мм-дд для хранения значения даты. Этот формат является фиксированным, и его невозможно изменить. (Для собственных форматов дат используйте DATE_FORMAT)

MySQL использует 3 байта для хранения значения DATE. Значения ДАТЫ варьируются от 1000-01-01 до 9999-12-31.

In [ ]:
connection.execute('''
CREATE TABLE people (
    id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    birth_date DATE NOT NULL
);
''')

In [ ]:
engine.table_names()

In [ ]:
pd.read_sql('''SELECT * from people''', connection)

Добавим элемент

In [ ]:
connection.execute('''
INSERT INTO people(first_name, last_name, birth_date)
VALUES('John','Doe','1990-09-01');
''')

Посмотрим на результат

In [ ]:
pd.read_sql('''
SELECT 
    id,
    first_name, 
    last_name, 
    birth_date
FROM
    people;
''', connection)

## Функции от дат

In [ ]:
!pip install mysql-connector-python

In [ ]:
import datetime
datetime.datetime.now() #текущее время

In [ ]:
pd.read_sql('''
SELECT CURRENT_TIMESTAMP
''', connection)

In [ ]:
pd.read_sql('''
SELECT DATE(CURRENT_TIMESTAMP) as `текущее время`;
''', connection)

In [ ]:
pd.read_sql('''
SELECT CURRENT_TIME
''', connection)

## Работа с триггером

Триггеры по своей сути представляют обработчики событий. Они выполняются при наступлении какого-либо простого действия в SQL. Такими действиями обычно являются: удаление, вставка и обновление данных.

То есть, триггер — это по сути ловушка, которая срабатывает при определенном действии. Триггер позволяет автоматизировать некоторые расчетные рутинные действия. Примеры мы разберем дальше.

Оператор для создания следующий:

`CREATE TRIGGER name_trigger`

После оператора и имени триггера необходимо указать в каком случае будет срабатывать триггер. Возможно 6 вариантов:

* BEFORE INSERT
* BEFORE UPDATE
* BEFORE DELETE
* AFTER INSERT
* AFTER UPDATE
* AFTER DELETE

In [ ]:
connection.execute(
'''
CREATE TABLE IF NOT EXISTS test_table(
    id int INTEGER AUTO_INCREMENT PRIMARY KEY NOT NULL,
    text TEXT NOT NULL
);
''')

In [ ]:
connection.execute('''
    CREATE TABLE IF NOT EXISTS log_data(
        id int AUTO_INCREMENT PRIMARY KEY,
        action text not null,
        table_name text not null
    );
''')

In [ ]:
connection.execute('''
    CREATE TRIGGER log_changes AFTER INSERT ON test_table
    FOR EACH ROW BEGIN
        INSERT INTO log_data (table_name, action) 
        VALUES ('test_table', 'add row');
    END;
''')

In [ ]:
connection.execute('''
insert into test_table(id, text) values (1, 'ttfhrtht')
''')

In [ ]:
pd.read_sql('Select * from test_table', connection)

In [ ]:
pd.read_sql('Select * from log_data', connection)

In [ ]:
connection.execute('''
insert into test_table(id, text) values (2, 'ttsrgrgsrh')
''')

In [ ]:
pd.read_sql('Select * from test_table', connection)

In [ ]:
pd.read_sql('Select * from log_data', connection)

Создание базы данных подписок по id

In [ ]:
connection.execute('''
CREATE TABLE IF NOT EXISTS subscriptions(
    subscriptionId INT AUTO_INCREMENT,
    userId INT,
    dailyEmail BOOLEAN NOT NULL DEFAULT FALSE,
    weeklyEmail BOOLEAN NOT NULL DEFAULT FALSE,
    specialOffer BOOLEAN NOT NULL DEFAULT FALSE,
    phone BOOLEAN NOT NULL DEFAULT FALSE,
    PRIMARY KEY (subscriptionId)
    );
''')

Создание базы данных пользователей с внешним ключем на список подписок

In [ ]:
connection.execute('''
CREATE TABLE IF NOT EXISTS users(
    userId INT AUTO_INCREMENT,
    subscription_id INT,
    firstName VARCHAR(255) NOT NULL,
    secondName VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL,
    emailConfimed BOOLEAN NOT NULL DEFAULT FALSE,
    PRIMARY KEY (userId),
    FOREIGN KEY (subscription_id)
        REFERENCES subscriptions(subscriptionId)
        ON UPDATE RESTRICT ON DELETE CASCADE
    );
''')

Загрузим пустую бд и удалим id (чтоб не получить ошибки автоинкремента)

In [ ]:
subscriptions = pd.read_sql('Select * from subscriptions', connection)
# дропаем столбец, который автоинкрементируется
del subscriptions['subscriptionId']
subscriptions.head()

добавим строку в конец датафрейма

In [ ]:
subscriptions.loc[len(subscriptions)] = [0, False, False, False, False]
subscriptions.head()

Добавим датафрейм в таблицу

In [ ]:
# добавляем в конец
subscriptions.to_sql('subscriptions', connection, if_exists='append', index=False)

In [ ]:
pd.read_sql('Select * from subscriptions', connection)


Создадим пользователя и удалим userId (чтоб не было ошибок автоинкремента)

In [ ]:
# создаем пользователя
users = pd.read_sql('Select * from users', connection)
# дропаем столбец, который автоинкрементируется
del users['userId']
users.head()

In [ ]:
users.loc[len(users)] = [1, 'Illya', 'Tykhov', 'tixons.work@gmail.com', True]
users.head()

Добавим датафрейм в таблицу sql

In [ ]:
users.to_sql('users', connection, if_exists='append', index=False)

In [ ]:
pd.read_sql('Select * from users', connection)

Агрегатная функция выполняет вычисление для нескольких значений и возвращает одно значение. Например, можно использовать агрегатную функцию AVG (), которая принимает несколько чисел и возвращает среднее значение чисел. Синтаксис:

`
function_name(DISTINCT | ALL expression)
`

In [ ]:
pd.read_sql('''
SELECT 
    COUNT(subscription_id) `количество`,
    AVG(subscription_id) `средняя величина`,
    MIN(subscription_id) `минимальная величина`,
    MAX(subscription_id) `максимальная величина`
FROM 
    users;
''', connection)

In [ ]:
pd.read_sql('''
SELECT 
    firstName `Имя`,
    COUNT(*) `Количество`
FROM
    users
GROUP BY subscription_id
ORDER BY subscription_id;
''', connection)

In [ ]:
# Пример интерфейса для работы с данными

In [ ]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')

cursor = conn.cursor()

In [ ]:
cursor.execute(
'''
CREATE TABLE user
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(255),
    lastName VARCHAR(255),
    age integer,
    deletedFlg integer
    )
'''
)
conn.commit()

In [ ]:
cursor.execute(
'''
INSERT INTO user(
    name,
    lastName,
    age,
    deletedFlg
)
VALUES ('Петя', 'Иванов', 31, 0)
'''
)
conn.commit()

In [ ]:
cursor.execute('SELECT * FROM user')
db_ =  cursor.fetchall()

for element in db_:
    print(element)

Добавим еще элементов 2умя способами

1 способ

In [ ]:
cursor.execute(
'''
INSERT INTO user(
    name,
    lastName,
    age,
    deletedFlg
)
VALUES ('Катя', 'Петрова', 30, 0),
        ('Аня', 'Воронцова', 25, 0),
        ('Дима', 'Петров', 35, 0)
'''
)
conn.commit()

In [ ]:
cursor.execute('SELECT * FROM user')
db_ =  cursor.fetchall()

for element in db_:
    print(element)

2 способ

In [ ]:
values = [
          ('Карина', 'Иванова', 26, 0),
          ('Вася', 'Воронцов', 28, 0),
          ('Света', 'Михайлова', 31, 0)
]

cursor.executemany('''
INSERT INTO user(
    name,
    lastName,
    age,
    deletedFlg
)
VALUES (?, ?, ?, ?)
''', values)
conn.commit()

In [ ]:
cursor.execute('SELECT * FROM user')
db_ =  cursor.fetchall()

for element in db_:
    print(element)

ЗАДАНИЕ

Как вывести результат в таблице pandas dataframe?

In [ ]:
pd.read_sql('SELECT * FROM user' , conn)

In [ ]:
# TODO

Пример интерфейса для дальнейшей работы с данными

In [ ]:
import sqlite3
import pandas as pd

class my_interface():
    # db_name - название таблицы
    def __init__(self, db_name):
        # создаем объект подключения с названием бд
        self.conn = sqlite3.connect(f'{db_name}.db')
        # операции с бд
        self.cursor = self.conn.cursor()
        
        # создание таблиц
        # таблица пользователей
        self.create_users()
        # таблица логирования
        self.create_log_info()
        
#         print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn) )
    
    # создание таблицы пользователей
    def create_users(self):
        try:
            self.cursor.execute(
            '''
            CREATE TABLE users
                (id INTEGER PRIMARY KEY AUTOINCREMENT,
                name VARCHAR(255),
                lastName VARCHAR(255),
                age integer,
                deletedFlg integer
                )
            '''
            )
        except Exception as e:
            print(e)
      
    # создание таблицы логирования
    def create_log_info(self):
        try:
            self.cursor.execute(
            '''
            CREATE TABLE loginfo
                (id INTEGER PRIMARY KEY AUTOINCREMENT,
                tablename VARCHAR(255) NOT NULL,
                action VARCHAR(255) NOT NULL,
                timestamp DATE DEFAULT (datetime('now','localtime'))
                )
            '''
            )
        except Exception as e:
            print(e)
      
    # логирование действий
    def log_info(self, tablename, action):
        self.cursor.execute(f'INSERT INTO loginfo(tablename, action) VALUES (\'{tablename}\', \'{action}\');')
        self.conn.commit()
      
    # логическое удаление из таблицы tablename по id
    def delete(self, tablename, id_, **kwargs):
        print(f'UPDATE {tablename} SET deletedFlg = 1 WHERE id = {id_}')
        self.cursor.execute(f'UPDATE {tablename} SET deletedFlg = 1 WHERE id = {id_}')
        self.log_info(tablename, 'delete')
        self.conn.commit()

    # отобразить данные из тыблицы tablename с ограничениями, описанными в kwargs
    def show_data(self, tablename, **kwargs):
        self.log_info(tablename, 'show')
        if kwargs != {}:
            if kwargs.get('limit') is not None:
                # возвращаем запрос с ограничениями на кол-во элементов, которые считываем из kwargs
                return pd.read_sql(f"SELECT * FROM {tablename} LIMIT {kwargs.get('limit')};", self.conn) 
        return pd.read_sql(f"SELECT * FROM {tablename}", self.conn) 
      
#     def add_users(self, values):
#         self.cursor.executemany('''
#         INSERT INTO users(    
#             name,
#             lastName,
#             age,
#             deletedFlg
#         )
#         VALUES (?, ?, ?, ?)
#         ''', values)
#         self.conn.commit()
    
    # добавление данных в таблицу tablename с полями, описанными в fields (list)
    def add_data(self, tablename, fields, values):
        # продумать интеграцию через to_sql библиотеки pandas
        # вставляем значения в таблицу, распаковываем названия полей и ? для подстановки данных
        # объединяем в одну строку суммированием
        self.cursor.executemany(
              "INSERT INTO " + 
              tablename + '(' + 
              ', '.join(fields) +
              ') VALUES (' + 
              ', '.join(['?' for i in range(len(values[0]))]) + ');'
        , values)
        self.conn.commit()
        self.log_info(tablename, 'add')


In [ ]:
values = [
          ('Аня', 'Иванова', 26, 0),
          ('Вася', 'Воронцов', 28, 0),
          ('Лена', 'Петрова', 31, 0)
        ]
        

ЗАДАНИЕ

1. Создать таблицу при помощи my_interface()

In [ ]:
# TODO

In [ ]:
interface = my_interface('my_interface')

In [ ]:
interface.show_data('users')

2. Добавить данные в таблицу `users` при помощи метода add_data()

In [ ]:
# TODO

In [ ]:
interface.add_data('users', ['name', 'lastName', 'age', 'deletedFlg'], values)

In [ ]:
interface.show_data('users')

3. Удалить пользователя с id = 1 при помощи delete()

In [ ]:
# TODO

In [ ]:
interface.delete('users', 1)

In [ ]:
interface.show_data('users')

4. Вывести логи по всем предыдущим операциям show_data('loginfo', limit = 5)

In [ ]:
interface.show_data('loginfo', limit = 5)

In [ ]:
# TODO

# ORM (объектно-реляционное отображение) на примере SQLalchemy

В этой части мы пройдем основные функции и подходы к созданию БД при помощи такого замечательного инструмента как SQLAlchemy :)

Переключим наше внимание на объектно-реляционное отображение. Первой целью является использование описанной нами системы таблиц метаданных для предоставления возможности переноса функций заданного пользователем класса на коллекцию столбцов в таблице базы данных. Второй целью является предоставление возможности описания отношений между заданными пользователем классами, которые будут основываться на отношениях между таблицами в базе данных.

В **SQLAlchemy** такая связь называется «отображением», что соответствует широко известному шаблону проектирования с названием «DataMapper»

## Классическое представление классов таблиц

Объект класса `sqlalchemy.orm.mapper.Mapper` связывает колонки из схемы таблицы и атрибуты Python класса.

In [ ]:
from sqlalchemy import Table, MetaData, Column, Integer, String, ForeignKey
from sqlalchemy.orm import mapper, relationship

metadata = MetaData()

user = Table('user', metadata,
             Column('id', Integer, primary_key=True),
             Column('name', String(50)),
             Column('fullname', String(50)),
             Column('password', String(12))
             )


address = Table('address', metadata,
                Column('id', Integer, primary_key=True),
                Column('user_id', Integer, ForeignKey('user.id')),
                Column('email_address', String(50))
                )


class User(object):
    pass


class Address(object):
    pass

print(dir(User))

mapper(
    User, user,
    properties={
        'addresses': relationship(Address, backref='user',
                                  order_by=address.c.id)
    })

print(dir(User))

mapper(Address, address)

## Декларативное представление классов таблиц

Каждый класс, представляющий таблицу в БД, должен наследоваться от базового класса который создается при помощи функции `sqlalchemy.ext.declarative.declarative_base().`

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()


In [ ]:
from sqlalchemy import Column, Integer, String

class User(Base):
     __tablename__ = 'user'

     id = Column(Integer, primary_key=True)
     name = Column(String)
     fullname = Column(String)

     def __repr__(self):
         return "<User(%r, %r)>" % (
                 self.name, self.fullname
             )

## Схема таблицы

Для каждого класса унаследованного от базового автоматически создается схема таблицы (объект класса sqlalchemy.schema.Table) и привязывается к нему через атрибут` __table__.`

In [ ]:
User.__table__

## MetaData

Любой класс таблицы автоматически ассоциируется с объектом sqlalchemy.schema.Table, который автоматически добавляется в список `sqlalchemy.schema.MetaData. `

Базовый класс `Base`, созданный при помощи функции `sqlalchemy.ext.declarative.declarative_base()`, является более высокоуровневой абстракцией над `sqlalchemy.schema.MetaData`, которая позволяет описывать таблицы декларативным способом. Таким образом все классы-таблицы имеют свою схему, которая хранится в атрибуте metadata базового класса Base:

In [ ]:
Base.metadata

In [ ]:
Base.metadata.tables.items()

Благодаря тому что Base содержит в себе объект `sqlalchemy.schema.MetaData`, вы можете пользоваться всеми его возможностями.

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://')
Base.metadata.create_all(engine)

## Mapper

Объект класса `sqlalchemy.orm.mapper.Mapper` связывает колонки из схемы таблицы и атрибуты из класса таблицы унаследованного от Base.

In [ ]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

Base = declarative_base()


class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column('password', String)

    addresses = relationship("Address", backref="user",
                             order_by="Address.id")


class Address(Base):
    __tablename__ = 'address'

    id = Column(Integer, primary_key=True)
    user_id = Column(ForeignKey('user.id'))
    email_address = Column(String)

address1 = Address(email_address="vas@example.com")
address2 = Address(email_address="vas2@example.com")
address3 = Address(email_address="vasya@example.com")

print("Mapper relationship: " + str(User.__mapper__.relationships))
print("Mapper columns: " + str(User.__mapper__.c.items()))

user1 = User(name="Вася")
user1.addresses = [address1, address2, address3]
print("User1 columns: " + str(user1.__table__.c.items()))
print(address1.user.name)

In [ ]:
engine = create_engine('sqlite://')
Base.metadata.create_all(engine)

## Конструктор класса

Декларативно описанный класс таблицы содержит в себе конструктор по умолчанию.

In [ ]:
ed_user = User(name='ed', fullname='Edward Jones', password = '123456')

Можно переопределить конструктор вручную

```
class User(Base):
    __tablename__ = 'user'

    def __init__(self, name, fullname):
       self.name = name
       self.fullname = fullname

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    addresses = relationship("Address", backref="user",
                             order_by="Address.id")
```

Поле `User.id` является первичным ключом, если его значение не указанно явно или такой id не существует в БД, то объект считается новым. После записи объекта в БД, значение поля id автоматически присваивается.

In [ ]:
print(ed_user.name, ed_user.fullname, ed_user.password)

In [ ]:
print(ed_user.id)

## Сессии

Сессии являются более абстрактным уровнем над механизмом соединения с СУБД `sqlalchemy.engine.Engine.` Они включают в себя функции хранения состояния объектов таблиц и записи этого состояния, по требованию, в БД.

См:

http://docs.sqlalchemy.org/en/latest/orm/session_basics.html

https://ru.wikibooks.org/wiki/SQLAlchemy/Sessions

Сессии создаются при помощи экземпляра класса `sqlalchemy.orm.session.Session.`

In [ ]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
session.add(ed_user)

In [ ]:
session.commit()

Перед выполнением любого запроса из сессии, состояние сессии автоматически переносится в БД. В нашем случае, не сохраненный объект `ed_user` добавляется в БД.

In [ ]:
our_user = session.query(User).filter_by(name='ed').first()

In [ ]:
our_user

In [ ]:
our_user.id

In [ ]:
ed_user.id

Что проверяет следующее условие?

In [ ]:
ed_user == our_user is ed_user

In [ ]:
session.add_all([
     User(name='wendy', fullname='Wendy Weathersmith'),
     User(name='mary', fullname='Mary Contrary'),
     User(name='fred', fullname='Fred Flinstone')
 ])

Если объект, находящийся в сессии, поменялся, то он помечается как `dirty.` Все измененные объекты в сессии доступны через атрибут `sqlalchemy.orm.session.Session.dirty`

In [ ]:
ed_user.fullname = 'Ed Jones'

In [ ]:
session.commit()

In [ ]:
for i in session.dirty:
    print(i)

Новые объекты, попавшие в сессию после ее сохранения или в новую сессию, доступны через атрибут `sqlalchemy.orm.session.Session.new`

In [ ]:
session.new

Метод `sqlalchemy.orm.session.Session.commit()` сохраняет состояние сессии в БД и подтверждает SQL транзакцию, в рамках которой выполнялись все предыдущие запросы.

In [ ]:
session.commit()

In [ ]:
ed_user.name

## SQL запросы через ORM

Операции над атрибутами класса таблицы равносильны операциям над объектом `sqlalchemy.schema.Column.` Поэтому их можно использовать в конструкторе SQL запросов. Результатом выполнения SQL выражения будет список значений записи в БД.

In [ ]:
print(User.name == "ed")

In [ ]:
from sqlalchemy import select
sel = select([User.name, User.fullname]).where(User.name == 'ed').order_by(User.id)

In [ ]:
session.connection().execute(sel).fetchall()

ORM позволяет конструировать запросы при помощи метода `sqlalchemy.orm.session.Session.query().` Этот метод создает объект класса `sqlalchemy.orm.query.Query`, который является более высокой абстракцией конструктора SQL выражения в SQLAlchemy.

ORM, в отличии от стандартного конструктора SQL выражения, позволяет создавать запросы более наглядно и возвращать результат в виде объектов которые привязаны к сессии.

In [ ]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)


Или комбинировать значения полей с объектами.

In [ ]:
for row in session.query(User, User.name):
    print(row.User.name,row.User.fullname, '...',row.name)

## Ограничения и условия

**LIMIT, OFFSET**

Выбор конкретной строки запроса делается не средствами языка Python, а на стороне СУБД, за счет конструкции `LIMIT ? OFFSET ?`, что значительно ускоряет выполнение запроса. Для программиста это выглядит прозрачно, как будто он работает с Python списком.

In [ ]:
u = session.query(User).order_by(User.id)[2]
print(u.fullname)

Аналогично работают и Python срезы.

In [ ]:
for u in session.query(User).order_by(User.id)[:]:
    print(u.fullname)

**WHERE**

Условие `WHERE` соответствует методу `sqlalchemy.orm.query.Query.filter_by().`

In [ ]:
for name in session.query(User.name).\
                 filter_by(fullname='Edward Jones'):
                 print(name)

Или более функциональному методу `sqlalchemy.orm.query.Query.filter()`

In [ ]:
for name, in session.query(User.name).\
                 filter(User.fullname == 'Edward Jones'):
                 print(name)

In [ ]:
from sqlalchemy import or_
for name, in session.query(User.name).\
                 filter(or_(User.fullname == 'Edward Jones', User.id < 5)):
                 print(name)

Последовательное выполнение методов `sqlalchemy.orm.query.Query.filter()` соединяет условия `WHERE` при помощи оператора `AND`, аналогично конструкции `select().where().`

In [ ]:
for user in session.query(User).\
                         filter(User.name == 'ed').\
                         filter(User.fullname == 'Ed Jones'):
                         print(user.name)

## Выполнение SQL выражений

Сам объект класса `sqlalchemy.orm.query.Query` не выполняет обращений к БД.

In [ ]:
query = session.query(User).filter_by(fullname='Ed Jones')

Для этого существуют специальные методы этого класса, например :

- `sqlalchemy.orm.query.Query.all()`
- `sqlalchemy.orm.query.Query.first()` - выполнит запрос и вернет первую строку запроса или None
- `sqlalchemy.orm.query.Query.one()` - выполнит запрос, вернет первую строку запроса и проверит что она одна и только одна, иначе вызовет исключение sqlalchemy.orm.exc.NoResultFound.


In [ ]:
for i in query.all():
    print(i.name)

In [ ]:
query.first().name

In [ ]:
query.one().name

In [ ]:
query = session.query(User).filter_by(fullname='nonexistent')
query.one()

In [ ]:
engine.table_names()

In [ ]:
Address.__table__.drop(engine)

In [ ]:
User.__table__.drop(engine)

In [ ]:
engine.table_names()